In [28]:
!pip install -q streamlit

In [29]:
import pandas as pd
import numpy as np
import nltk
import string
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import gensim
from gensim.models import Word2Vec

In [30]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [31]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [32]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [33]:
df= pd.read_csv('/content/drive/MyDrive/Datasets/Myntra Fasion Clothing.csv')
df.head()

<ipython-input-33-5c588748e2d8>:1: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df= pd.read_csv('/content/drive/MyDrive/Datasets/Myntra Fasion Clothing.csv')


,URL,Product_id,BrandName,Category,Individual_category,category_by_Gender,Description,DiscountPrice (in Rs),OriginalPrice (in Rs),DiscountOffer,SizeOption,Ratings,Reviews
0,https://www.myntra.com/jeans/roadster/roadster...,2296012,Roadster,Bottom Wear,jeans,Men,roadster men navy blue slim fit mid rise clean...,824.0,1499.0,45% OFF,"28, 30, 32, 34, 36",3.9,999.0
1,https://www.myntra.com/track-pants/locomotive/...,13780156,LOCOMOTIVE,Bottom Wear,track-pants,Men,locomotive men black white solid slim fit tra...,517.0,1149.0,55% OFF,"S, M, L, XL",4.0,999.0
2,https://www.myntra.com/shirts/roadster/roadste...,11895958,Roadster,Topwear,shirts,Men,roadster men navy white black geometric print...,629.0,1399.0,55% OFF,"38, 40, 42, 44, 46, 48",4.3,999.0
3,https://www.myntra.com/shapewear/zivame/zivame...,4335679,Zivame,Lingerie & Sleep Wear,shapewear,Women,zivame women black saree shapewear zi3023core0...,893.0,1295.0,31% OFF,"S, M, L, XL, XXL",4.2,999.0
4,https://www.myntra.com/tshirts/roadster/roadst...,11690882,Roadster,Western,tshirts,Women,roadster women white solid v neck pure cotton ...,NaN,599.0,35% OFF,"XS, S, M, L, XL",4.2,999.0


In [34]:
df.shape

(526564, 13)

In [35]:
# If want to include all of product description:  description +BrandName+Category+...
#df['raw_text'] = df['BrandName'] + ' '+ df['Category'] +' '+ df['Individual_category'] +' '+ df['category_by_Gender'] +' '+ df['Description']
#df['raw_text']

In [36]:
def preprocess_txt(text):
  # convert all words to lower case
  text = text.lower()
  # remove all punctuations 
  text_p = "".join([char for char in text if char not in string.punctuation])
  # tokenize the sentence
  words = word_tokenize(text_p)
  # remove stop words of english corpora
  stop_words = stopwords.words('english')
  filtered_words = [word for word in words if word not in stop_words]
  # remove duplicate words
  final_text = list(set(filtered_words))
  return final_text

In [37]:
# process the product description column and store it in a new column
df['processed_text'] = [' '.join(word for word in preprocess_txt(s)) for s in list(df['Description'])]
df['processed_text']

0         mid clean jeans look roadster fit men rise sli...
1         track solid fit pants men white slim black loc...
2         sustainable printed casual roadster men white ...
3         saree women shapewear zivame black zi3023core0...
4         neck women cotton pure roadster solid v white ...
                                ...                        
526559             jumper hm women beige oversized cashmere
526560          water repellent hm women jacket black parka
526561                   sheath designs 250 lace dress blue
526562               printed women masaba line dress yellow
526563             extended sleeves jockey women blue shirt
Name: processed_text, Length: 526564, dtype: object

In [38]:
sentences = list(df['processed_text'].values)

**Doc2Vec**

In [39]:
from gensim.models import doc2vec
from collections import namedtuple

docs = []
# create tuples of sentences containg the sentence and its index, and store them in docs
analyzedDocument = namedtuple('AnalyzedDocument', 'words tags')
for i, text in enumerate(sentences):
    words = text.lower().split()
    tags = [i]
    docs.append(analyzedDocument(words, tags))

# Train model (set min_count = 1, if you want the model to work with the provided example data set)

model = doc2vec.Doc2Vec(docs, vector_size = 100, window = 5, min_count = 1)

In [40]:
# Visualize our embeddings
model.dv[0]

array([ 0.0413686 ,  0.00165472,  0.01808582,  0.06252225,  0.01978071,
       -0.01674396,  0.0092371 , -0.01497122, -0.02467414,  0.00652668,
       -0.00363385, -0.03850319, -0.0017023 , -0.00974618, -0.04461294,
       -0.0364798 ,  0.03819523,  0.01710616,  0.00735543, -0.01120766,
       -0.03635309,  0.04315861, -0.01431848, -0.01352224,  0.0354408 ,
       -0.01499189, -0.0373788 , -0.01244939,  0.06822794,  0.02791616,
        0.00193861,  0.00274997,  0.00230948, -0.04152346, -0.04624591,
        0.00735034, -0.00614942,  0.08729645, -0.04388147, -0.01851941,
        0.00462456, -0.06897203,  0.02223651, -0.07451073, -0.07112597,
        0.01017205, -0.00094792, -0.01312823, -0.01737777,  0.03142688,
       -0.0306346 , -0.0081484 ,  0.02837589,  0.03394856, -0.01046773,
       -0.01634424,  0.03338401,  0.01853191, -0.03037832,  0.10851377,
       -0.00795471, -0.03078097, -0.04333522,  0.0040378 ,  0.01313747,
       -0.03148974,  0.00835421, -0.01562976,  0.00061412, -0.02

In [41]:
# save the model
from gensim.test.utils import get_tmpfile
model_name = get_tmpfile("/content/my_doc2vec_model")
model.save(model_name)

In [42]:
# append the respective embeddings into the dataframe
df['doc2vec'] = [list(model.dv[i]) for i in range(df.shape[0])]

In [43]:
# input a target text and convert it to embeddings
vector = model.infer_vector(["I need a blue jeans"])
vector

array([-0.00176919,  0.00199916,  0.0001458 , -0.00318866, -0.00320263,
       -0.00393302, -0.00463751, -0.00377058,  0.00450507,  0.00067582,
       -0.00239396, -0.00073987,  0.00431561,  0.00056175,  0.00302343,
        0.00402881, -0.00384925,  0.00336534, -0.00236951,  0.00456594,
        0.00316771,  0.00083225, -0.00223422,  0.0009445 ,  0.00247634,
       -0.00465292, -0.0033156 ,  0.00020113,  0.00161084,  0.00449724,
       -0.0021251 , -0.00204079, -0.00463605, -0.00099744, -0.00059226,
       -0.00142908, -0.00149552,  0.00199352, -0.0036052 ,  0.00367415,
       -0.00353515, -0.00457998,  0.003543  , -0.00190902,  0.00086711,
       -0.00247238, -0.00428162, -0.00060012, -0.00270851, -0.00492436,
        0.00347412,  0.00340929, -0.00443253,  0.00495183,  0.00235306,
       -0.0026725 , -0.00387374, -0.00325281, -0.00205865,  0.00138897,
       -0.00482885, -0.00131069,  0.00044426, -0.00055432, -0.00040387,
       -0.00318618,  0.00351528, -0.00204405,  0.00261635, -0.00

In [ ]:
# save the dataframe 
df.to_csv('/content/drive/MyDrive/Embeddings.csv')

In [18]:
def cos_similarity(v1,v2):
  return abs(np.dot(v1,v2))/(np.linalg.norm(v1)*np.linalg.norm(v2))

In [19]:
def ranked_results(df, target, model):
  # target needs to be preprocessed in the same way as the product descriptions were processed
  if isinstance(target,str):
    target = preprocess_txt(target)
  #convert the processed target into embeddings
  vector = model.infer_vector(target)
  df_ = df[['URL','processed_text','doc2vec']]
  # store the cosine similarity scores for each product with the target in a new column
  df_['similarity'] = [cos_similarity(vector, df_['doc2vec'][i]) for i in range(df_.shape[0])]
  # sort the similarity scores in descending order
  df_.sort_values(by='similarity', ascending = False, inplace=True)
  return df_[['URL','processed_text','similarity']].iloc[:10]


In [20]:
target = "Blue Jeans for women"
ranked_results(df, target, model)

<ipython-input-19-a9d3a195956c>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_['similarity'] = [cos_similarity(vector, df_['doc2vec'][i]) for i in range(df_.shape[0])]
<ipython-input-19-a9d3a195956c>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_.sort_values(by='similarity', ascending = False, inplace=True)


,URL,processed_text,similarity
498011,https://www.myntra.com/jeans/showoff/showoff-w...,straight stretchable women jeans high fit show...,0.879372
24530,https://www.myntra.com/briefs/amante/amante-wo...,2 amante briefs women pack,0.856998
240696,https://www.myntra.com/socks/bambos/bambos-men...,3 bambos calf length men pack socks patterned,0.856623
64041,https://www.myntra.com/jeans/flying-machine/fl...,stretchable clean michael jeans look fit men r...,0.846804
410696,https://www.myntra.com/nightdress/erotissch/er...,kaftan nightdress printed erotissch blue,0.846415
476873,https://www.myntra.com/jeans/tarama/tarama-wom...,leg women jeans high wide tarama rise blue,0.843426
11131,https://www.myntra.com/jeans/mast--harbour/mas...,stretchable boyfriend women jeans fit mast har...,0.843191
521607,https://www.myntra.com/jeans/only/only-women-b...,women jeans high rise black,0.842505
449139,https://www.myntra.com/tops/scorpius/scorpius-...,crop scorpius top fitted blue,0.842384
421114,https://www.myntra.com/shapewear/wuxi/wuxi-wom...,saree women solid shapewear wuxi black,0.841778


In [21]:
%%writefile app.py
import streamlit as st


def main():

    import pandas as pd
    import numpy as np
    import nltk
    import string
    from nltk import word_tokenize
    from nltk.corpus import stopwords
    import gensim
    from gensim.models import doc2vec
    import ast
    import json



    def preprocess_txt(text):
      # convert all words to lower case
      text = text.lower()
      # remove all punctuations 
      text_p = "".join([char for char in text if char not in string.punctuation])
      # tokenize the sentence
      words = word_tokenize(text_p)
      # remove stop words of english corpora
      stop_words = stopwords.words('english')
      filtered_words = [word for word in words if word not in stop_words]
      # remove duplicate words
      final_text = list(set(filtered_words))
      return final_text



    def cos_similarity(v1,v2):
      return abs(np.dot(v1,v2))/(np.linalg.norm(v1)*np.linalg.norm(v2))



    def ranked_results(df, target, model):
    # target needs to be preprocessed in the same way as the product descriptions were processed
      if isinstance(target,str):
        target = preprocess_txt(target)
      #convert the processed target into embeddings
      vector = model.infer_vector(target)
      df_ = df[['URL','processed_text','doc2vec']]
      # store the cosine similarity scores for each product with the target in a new column
      df_['similarity'] = [cos_similarity(vector, ast.literal_eval(df_['doc2vec'][i])) for i in range(df_.shape[0])]
      # sort the similarity scores in descending order
      df_.sort_values(by='similarity', ascending = False, inplace=True)
      url_dict = df_[['URL']].iloc[:10].to_dict()
      json_object = json.dumps(url_dict, indent=4)
      return json_object



    df = pd.read_csv('/content/drive/MyDrive/Embeddings.csv')
    model = doc2vec.Doc2Vec.load('/content/my_doc2vec_model')


    st.title("Streamlit Similar Clothing Recomendations App")
    html_temp = """
    <div style="background-color:tomato;padding:10px">
    <h2 style="color:white;text-align:center;">"You may also like these!"</h2>
    </div>
    """
    st.markdown(html_temp,unsafe_allow_html=True)
    target = st.text_input("Input Text","Type Here")
    result=""
    if st.button("Search"):
        result=ranked_results(df, target, model)
    st.success('The output is {}'.format(result))

if __name__=='__main__':
    main()

Writing app.py


In [22]:
!npm install localtunnel

npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm notice created a lockfile as package-lock.json. You should commit this file.
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ localtunnel@2.0.2
added 22 packages from 22 contributors and audited 22 packages in 2.353s

3 packages are looking for funding
  run `npm fund` for details

found 0 vulnerabilities



In [23]:
!streamlit run /content/app.py &>/content/logs.txt &

In [24]:
!curl ipecho.net/plain

34.125.112.200

In [25]:
!npx localtunnel --port 8501

npx: installed 22 in 3.657s
your url is: https://happy-times-flash.loca.lt
/root/.npm/_npx/3479/lib/node_modules/localtunnel/bin/lt.js:81
    throw err;
    ^

Error: connection refused: localtunnel.me:43155 (check your firewall settings)
    at Socket.<anonymous> (/root/.npm/_npx/3479/lib/node_modules/localtunnel/lib/TunnelCluster.js:52:11)
    at Socket.emit (events.js:315:20)
    at emitErrorNT (internal/streams/destroy.js:106:8)
    at emitErrorCloseNT (internal/streams/destroy.js:74:3)
    at processTicksAndRejections (internal/process/task_queues.js:80:21)


In [27]:
df1 = pd.read_csv('/content/drive/MyDrive/Embeddings.csv')
model1 = doc2vec.Doc2Vec.load('/content/my_doc2vec_model')
target = "Blue Jeans for women"
ranked_results(df1, target, model1)

<ipython-input-27-caf1e0a4eb5b>:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv('/content/drive/MyDrive/Embeddings.csv')


UFuncTypeError: ignored

In [ ]:
import ast
target = "Blue Jeans for women"
vector = model.infer_vector(preprocess_txt(target))
cos_similarity(ast.literal_eval(df1 ['doc2vec'][0]),vector)